# Estrapolazione dati aggiornati delle nazioni dai file html
- Con l'ausilio della libreria spacy di NLP procederemo ad estrapolare i dati dal 2013 in poi per le nazioni.

In [2]:
# importazione dei pacchetti utili

import spacy 
import spacy_transformers
from spacy.language import Language
import pandas as pd
import pickle

In [5]:
# carico la lista dei country

infile = open("Data/country_list_viz.pickle",'rb')
country_list = pickle.load(infile)
infile.close()

In [1]:
country_list

In [35]:
# occorre eliminare gli spazi nei nomi ed inserire i trattini -, definisco inoltre un correction_dict per risistemare i nomi

country_list_correct=set()

correction_dict={}

for country in country_list:
    
    country_list_string=list(country)
    
    for i in range(len(country_list_string)):
        
        if country_list_string[i]== ' ':
            
            country_list_string[i]='-'
        
    country_string=''.join(country_list_string).lower()
    
    country_list_correct.add(country_string)
    
    correction_dict[country_string]=country
    

In [2]:
# caricamento english pipeline spacy

nlp = spacy.load("en_core_web_sm")

In [3]:
# aggiungo pipeline per andare a capo quando si incontra \n 

@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == '\n  ':
            doc[token.i].is_sent_start = True
    return doc

nlp.add_pipe("set_custom_boundaries", before="parser")

<function __main__.set_custom_boundaries(doc)>

In [6]:
def get_data(nation):
    
    #inizializzo variabili
    
    f=open('Data/GlobalTemp_countries_dataset/{}.html'.format(nation))
    j=0
    a=f.read()
    clean_data_list=[]
    control_list=[]
    l=[]
    reference_temp=[]
    reference_temp_2=[]
    clean_data_list_2=[]
    
    #tokenizzo documento
    
    doc = nlp(a)
    
    #creo lista con gli elementi del file txt
    
    for token in doc:
        
        l.append(token.text)

    #ricavo le reference temp
    
    try:
        
        for n in range(len(l)):

            control = [l[n],l[n+1],l[n+2],l[n+3]]

            if control == ['Dec','\n','%','%']:
                
                reference_temp=l[n+4:n+28]

                break
                
    except IndexError:
        
            for n in range(len(l)):

                control = [l[n],l[n+1],l[n+2]]

                if control == ['Dec','\n','%']:
                    
                    reference_temp=l[n+3:n+27]

                    break
                    
                    
    while len(reference_temp_2)<12:
        
        if reference_temp[j] not in {' '*k for k in range(10)} :
            
            reference_temp_2.append(reference_temp[j])
            
        j+=1
            
    #considero le ultime 110 righe da cui estrapolare i dati relativi al 2013
    
    data_sent=list(doc.sents)[len(list(doc.sents))-110:]    
    data_list=[]
    
    for n,sent in enumerate(data_sent):
                
        if str(sent).split()[0] == '2013':
            
            data_sent=data_sent[n:]
            
            break    
    
    for i,data in enumerate(data_sent):

        data_list.append(str(data).split())
        
        if len(data_list[i])>12:
        
            for item in data_list[i]:

                if item in ['</pre></body></html>','</pre></body></html', '>'] :

                    data_list[i].remove(item) 
                    
    for llist in data_list:
        
        if llist == ['>'] or llist==[]:
            
            data_list.remove(llist)
            
    for k,llist in enumerate(data_list):
        
        if len(llist)<12 and llist not in control_list :
            
            control_list.append(llist)
            control_list.append(data_list[k+1])            
            clean_data_list.append(llist+data_list[k+1])
            
        elif data_list[k] in clean_data_list:
            
            continue
            
        elif llist in control_list:
            
            continue
        
        else:
            
            clean_data_list.append(data_list[k])
            
            
    for h,llist in enumerate(clean_data_list):
        
        if len(llist)>12 and int(llist[0]) in range(1,13):
            
            correction=[]
            
            if llist[0] != '1':
                                
                llist.insert(0,clean_data_list[h-1][0])
                
            else:
                                
                llist.insert(0,clean_data_list[h+1][0])
                
            clean_data_list_2.append(llist[0:12])
            clean_data_list_2.append(llist[12:24])
            
        else:
            
            clean_data_list_2.append(llist)
    
    
    for h,llist in enumerate(clean_data_list_2):

        if int(llist[0]) not in list(range(2013,2021)):

                if str(llist[11]) != 'NaN':

                    if int(llist[11]) in list(range(2013,2021)):

                        v=llist[11]
                        llist.pop()

                        llist.insert(0,v)

                elif int(llist[0]) in list(range(1,13)) and int(llist[1]) in list(range(1,13)):

                    if llist[1] != '1':

                        llist[0]=clean_data_list[h-1][0]

                    else:

                        llist[0]=str(int(clean_data_list[h-1][0])+1)        
     
        
    return clean_data_list_2, list(map(float,reference_temp_2))

In [7]:
#ricava le temperature medie dalle anomalie estrapolate con get_data

def get_temp(data_list,reference_temp):
    
    temp_list=[]
    
    for data in data_list:
        
        temp_list.append( [data[0], data[1], float(data[2])+reference_temp[int(data[1])-1]] )
    
    return temp_list

In [8]:
dt=[]
temp_list=[]
c_list=[]
c_errors=[]
i=0

for country in country_list_correct:
    
    try:
        (a,b) = get_data(country)
        temp_data=get_temp(a,b)

        for item in temp_data:

            if int(item[1])<10:

                dt.append(item[0]+'-0'+item[1]+'-01')
                temp_list.append(item[2])
                c_list.append(country)
            else:

                dt.append(item[0]+'-'+item[1]+'-01')
                temp_list.append(item[2])
                c_list.append(country)
                
        print(i, end= ' - ')
        i+=1
                
    except:
        
        print(country,end=' - ')
        c_errors.append(country)
        
            
        
dataset_dict={'dt':dt,'AverageTemperature':temp_list,'Country':c_list}

df_update=pd.DataFrame.from_dict(dataset_dict)

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 10 - 11 - 12 - 13 - 14 - 15 - 16 - 17 - 18 - 19 - 20 - 21 - 22 - 23 - 24 - 25 - 26 - 27 - 28 - 29 - 30 - 31 - 32 - 33 - 34 - 35 - 36 - 37 - côte-d'ivoire - 38 - 39 - 40 - 41 - 42 - 43 - 44 - 45 - 46 - 47 - 48 - 49 - 50 - 51 - 52 - 53 - 54 - 55 - 56 - 57 - 58 - 59 - 60 - 61 - 62 - 63 - 64 - 65 - 66 - 67 - 68 - 69 - 70 - 71 - 72 - 73 - 74 - 75 - 76 - 77 - 78 - 79 - 80 - 81 - 82 - 83 - 84 - 85 - 86 - 87 - 88 - 89 - 90 - 91 - 92 - 93 - 94 - 95 - 96 - 97 - 98 - 99 - 100 - 101 - 102 - 103 - 104 - 105 - 106 - 107 - 108 - 109 - 110 - 111 - 112 - 113 - 114 - 115 - 116 - 117 - 118 - 119 - 120 - 121 - 122 - 123 - 124 - 125 - 126 - 127 - 128 - 129 - 130 - 131 - 132 - 133 - 134 - 135 - 136 - 137 - 138 - 139 - 140 - 141 - 142 - 143 - 144 - 145 - 146 - 147 - 148 - 149 - 150 - 151 - 152 - 153 - 154 - 155 - 156 - 157 - 158 - 159 - 160 - 161 - 162 - 163 - 164 - 165 - 

In [16]:
#per la costa d'avorio non sono presenti dati aggiornati 

c_errors

["côte-d'ivoire"]

In [37]:
df_update_1=df_update

In [38]:
df_update_1['Country']=df_update_1['Country'].replace(correction_dict)
df_update_1

,dt,AverageTemperature,Country
0,2013-01-01,1.940,Afghanistan
1,2013-02-01,4.718,Afghanistan
2,2013-03-01,10.870,Afghanistan
3,2013-04-01,14.751,Afghanistan
4,2013-05-01,21.327,Afghanistan
...,...,...,...
15271,2020-08-01,19.621,Zimbabwe
15272,2020-09-01,22.638,Zimbabwe
15273,2020-10-01,24.588,Zimbabwe
15274,2020-11-01,25.934,Zimbabwe


In [39]:
df_update_1.to_csv('Data/aggiornamento_dati_nazioni_1.csv')